# Introduction to Cython
Part of this lecture is based on the material by [Dr. Gregory Watson](https://nyu-cds.github.io/python-itertools/)

This lecture provides a very brief introduction to Cython. See the [Cython documentation](http://cython.readthedocs.io/en/latest/) for a more detailed description of the Cython language.

What we will learn:
- What is cython
- The different manners of using cython
- some comparison in terms of computational times

----
Cython is a modification of Python that adds C data types and converts python codes to C, thus allowing for compilation into a shared library that can be imported into Python. Almost any piece of Python code is also valid Cython code (with a few limitations).

In Cython, function parameters and variables can be declared to have C data types and code which manipulates Python values and C values can be freely intermixed. Cython takes care of automatically converting from C to Python data types wherever possible.

__Speedup__<br>

- Depends very much on the program
- Python numerical programs tend to gain little as most time is spent in lower-level C anyway. 
- For-loop-style programs can improve by many orders of magnitude.


<blockquote>
<h2 id="prerequisites"><font color='blue'>__Prerequisites__</font></h2>
    <p>The examples in this lesson can be run directly using the Python interpreter, using IPython interactively, 
or using Jupyter notebooks. Anaconda users will already have Cython installed. You will also need a functioning
C compiler to be able to use Cython. See the <a href="http://cython.readthedocs.io/en/latest/src/quickstart/install.html">Cython installation guide</a> for more details.</p>
</blockquote>

#### <font color='blue'>Basic C Types</font>
| Type        |	Description |
| :---        | :---: |
| char	| 8-bit signed integer |
| short	| 16-bit signed integer |
|int	| 32-bit signed integer |
| long	| 64-bit signed integer |
| long long	| 64-bit signed integer|
| float	| 32-bit floating point |
| double |64-bit floating point |
| long double | 80-bit floating point |<br>
#### <font color='blue'>Array</font>
type name[size]
#### <font color='blue'>Pointer</font>
type *name
#### <font color='blue'>Structure</font>
struct name { declaration }

----
### Using the magic %%cython in jupyter

In [1]:
%load_ext Cython

In [2]:
%%cython

cdef int a=0
cdef float g[10]

for i in range(10):
    g[i]=a
    a += i
    
print(g)

[0.0, 0.0, 1.0, 3.0, 6.0, 10.0, 15.0, 21.0, 28.0, 36.0]


In [10]:
%%cython --annotate

cdef int a=0
cdef float g[10]

for i in range(10):
    g[i]=a
    a += i
    
print(g)

[0.0, 0.0, 1.0, 3.0, 6.0, 10.0, 15.0, 21.0, 28.0, 36.0]


In [3]:
%%cython

cdef struct Register:
    unsigned char *name
    int age
    float weight
    
cdef Register my_data

my_data.name = 'Luis'
my_data.age = 48
my_data.weight = 180

print(my_data)

{'name': b'Luis', 'age': 48, 'weight': 180.0}


----
### Using cython withoug jupyter (Compiling with distutils)

Cython code is normally saved in files ending with .pyx (the x indicates it is different from standard Python code). A Cython file can be translated to C using the **distutils** package.

The **distutils** package is part of the standard library. It is the standard way of building Python packages, including native extension modules. The following example configures the build for a Cython file called **my_module.pyx** with the content:
```python
def cfunc(int n):
    cdef int a = 0
    cdef int i
    for i in range(n):
        a += i
    return a
```

In order to use **distutils** we have to create a **setup.py** script. In our example it can be:

```python
from distutils.core import setup
from Cython.Build import cythonize

setup(
    name = "My module app",
    ext_modules = cythonize('my_module.pyx'), 
)
```
Now, run the command 
```shell
python setup.py build_ext --inplace
``` 
in your system’s command shell and you are done.

The two files:
- my_module.c
- my_module.cpython-36m-darwin.so
will be created

The .so library can be treated just like any Python module and imported using the normal import statement:
```python
import my_module
```

In [6]:
import my_module

a = my_module.cfunc(100)
print(a)

4950


If you have to include files in non-standard places you can pass an include_path parameter to cythonize. For instance, using numpy we have to change the row in setup.py to:
```python
ext_modules = cythonize('my_module.pyx',
                        include_path = [numpy.get_include()]),
```

In [1]:
import my_module_with_numpy as mmwn

A = mmwn.cfunc_with_numpy(10)
print(A)

[[ 1013.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.  1013.     0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.  1013.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.  1013.     0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.  1013.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.  1013.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.  1013.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.  1013.     0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.  1013.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.  1013.]]


----
### Using pyximport

Cython also provides a means of importing Cython programs directly using the **pyximport** module. To use pyximport, you must first import and initialize the module, then your programs will be able to use the import statement as you would normaly do for a Python module. The pyxmodule is included with Cython.
```python
import pyximport
pyximport.install()
```
```python
import my_cython_module as mcm
```

----
#### Performance Comparisons
The following pure Python example generates a list of kmax prime numbers

In [1]:
# Pure Python code

import time

def primes(kmax):
    p = [None] * 1000 # Initialize the list to the max number of elements
    if kmax > 1000:
        kmax = 1000
    result = []
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

t = time.process_time()
x = primes(1000)
elapsed_time = time.process_time() - t
print(elapsed_time,'s')

0.07078400000000007 s


In [8]:
%load_ext Cython

In [12]:
%%cython
# Using the magic cython

import time

def primes_with_cython(kmax):
    p = [None] * 1000 # Initialize the list to the max number of elements
    if kmax > 1000:
        kmax = 1000
    result = []
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

t = time.process_time()
x = primes_with_cython(1000)
elapsed_time = time.process_time() - t
print(elapsed_time,'s')

In [4]:
# externally compiled

import time 
import external_prime_module as epm

t = time.process_time()
x = epm.external_primes_with_cython(1000)
elapsed_time = time.process_time() - t
print(elapsed_time,'s')

0.04518299999999997 s


In [2]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x1099d4160>)

In [11]:
# using pyximport

import time 
import external_prime_module_without_compiling as epmwc

t = time.process_time()
x = epmwc.external_primes_with_cython(1000)
elapsed_time = time.process_time() - t
print(elapsed_time,'s')

0.043477999999999906 s
